<a href="https://colab.research.google.com/github/uo294258/ri/blob/main/Ejercicio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install bm25s[full]
import bm25s # To create indexes and search documents using BM25
import Stemmer  # For stemming terms
import json  # To load the JSON-formatted corpus
import csv

!gdown 1q_8DhdlF9imN-ksNWAjrT-HpeO7wFR-4
!unzip -o trec-covid-RI.zip
corpus_content = []

with open("corpus.jsonl", "r", encoding="utf-8") as f:
    for line in f:
          json_object = json.loads(line)
          corpus_content.append(json_object)
corpus_verbatim = list()
corpus_plaintext = list()
for entry in corpus_content:
    document = {"id": entry["_id"], "title": entry["title"], "text": entry["text"].lower()}
    corpus_verbatim.append(document)
    corpus_plaintext.append(entry["text"].lower())

stemmer = Stemmer.Stemmer("english")



Downloading...
From (original): https://drive.google.com/uc?id=1q_8DhdlF9imN-ksNWAjrT-HpeO7wFR-4
From (redirected): https://drive.google.com/uc?id=1q_8DhdlF9imN-ksNWAjrT-HpeO7wFR-4&confirm=t&uuid=d6a403dc-07b9-465c-a7ab-28b40037d29e
To: /content/trec-covid-RI.zip
100% 71.7M/71.7M [00:00<00:00, 95.3MB/s]
Archive:  trec-covid-RI.zip
  inflating: 3451964.3451965.pdf     
  inflating: corpus.jsonl            
 extracting: LEEME.txt.txt           
  inflating: qrels.tsv               
  inflating: queries.jsonl           


In [9]:
all_queries = dict()
filename="queries.jsonl"

all_queries[filename] = []
with open(filename, "r", encoding="utf-8") as f:
      for line in f:
          data = json.loads(line)
          all_queries[filename].append(data)
print(all_queries)

{'queries.jsonl': [{'_id': '1', 'text': 'what is the origin of COVID-19', 'metadata': {'query': 'coronavirus origin', 'narrative': "seeking range of information about the SARS-CoV-2 virus's origin, including its evolution, animal source, and first transmission into humans"}}, {'_id': '2', 'text': 'how does the coronavirus respond to changes in the weather', 'metadata': {'query': 'coronavirus response to weather changes', 'narrative': 'seeking range of information about the SARS-CoV-2 virus viability in different weather/climate conditions as well as information related to transmission of the virus in different climate conditions'}}, {'_id': '3', 'text': 'will SARS-CoV2 infected people develop immunity? Is cross protection possible?', 'metadata': {'query': 'coronavirus immunity', 'narrative': 'seeking studies of immunity developed due to infection with SARS-CoV2 or cross protection gained due to infection with other coronavirus types'}}, {'_id': '4', 'text': 'what causes death from Covi

In [10]:
relevance_judgements = dict() # Inicializar un diccionario vacío

with open("qrels.tsv", "r", encoding="utf-8") as f:
    reader = csv.reader(f, delimiter="\t")  # Crear un lector TSV
    next(reader)  # Saltar la primera línea con encabezados ("query-id corpus-id score")

    for row in reader:
        query_id, doc_id, relevance = row  # Desempaquetar valores de la fila

        # Agregar la información al diccionario
        if query_id not in relevance_judgements:
            relevance_judgements[query_id] = {}
        relevance_judgements[query_id][doc_id] = int(relevance)

print(relevance_judgements)

{'1': {'005b2j4b': 2, '00fmeepz': 1, 'g7dhmyyo': 2, '0194oljo': 1, '021q9884': 1, '02f0opkr': 1, '047xpt2c': 0, '04ftw7k9': 0, 'pl9ht0d0': 0, '05vx82oo': 0, '0604jed8': 0, '06o2tbon': 0, '06ya15z8': 0, '084o1dmp': 0, '08ds967z': 1, '08efpohc': 0, '0b4o0ccp': 0, 'brqby02y': 2, '0chuwvg6': 2, '0cvoeiy0': 0, '0e1w86tg': 1, 'qotm49rv': 1, '0hnh4n9e': 1, '0iq9s94n': 1, '0khg28ex': 0, '0l33i6s4': 0, '0lyxvex0': 0, '0m5mc320': 0, 'ex7rta8f': 2, '0nh58odf': 2, '0oiq44gl': 0, '0paafp5j': 0, '0pbjttv4': 0, '0qaoam29': 0, '0qpfoh5t': 0, '0t2a5500': 2, '0ti403i4': 0, '0v5wo0ty': 1, '0xhho1sh': 2, '0xruezf2': 1, '0y34yxlb': 2, '105q161g': 2, '10ecm4wi': 1, '11edrkav': 2, '127c5bve': 2, '12dcftwt': 2, '13ir7swr': 2, '13jupb26': 0, '1585stal': 0, 'cysldr06': 0, '16rgt4ca': 0, '18xs6375': 0, '1a8uevk8': 0, '1aal6njl': 0, '1abp6oom': 2, 'pe8qxorf': 0, '1bapn9w0': 0, '1bpc8g6n': 0, '1bvsn9e8': 2, '1c47w4q5': 1, '1de98sxz': 0, '1dogron2': 1, 'yggdch81': 0, '1esupl4q': 0, '1eytelxn': 1, '1g2mup0k': 0, '1h

In [11]:
def submit_queries_and_get_run(filename,queries, stemmer, retriever, max_results=100):
    run = {}
    for query in queries:
        query_id = query["_id"]

        query_string = query["text"].lower()

        query_tokenized = bm25s.tokenize(query_string, stopwords="en", stemmer=stemmer, show_progress=False)

        results = retriever.retrieve(query_tokenized, corpus=retriever.corpus, k=max_results, return_as="tuple", show_progress=False)

        returned_documents = results.documents[0]
        relevance_scores = results.scores[0]

        returned_ids = []
        for i in range(len(returned_documents)):
##            print(f"\t{i}\t{relevance_scores[i]}\t{returned_documents[i]['id']}\t{returned_documents[i]['title'][0:80]}...")
            returned_ids.append(str(returned_documents[i]["id"]))
        run[query_id] = returned_ids

    with open(filename, "w", encoding="utf-8") as f:
        json.dump(run, f, indent=4)

    return run

In [12]:
def compute_precision_recall_f1(run, relevance_judgements):
    # Initialize lists to hold precision, recall, and f1 scores for each query
    precision_values = []
    recall_values = []
    f1_values = []

    # Initialize global counts for micro-averaging
    global_retrieved = 0
    global_relevant = 0
    global_retrieved_and_relevant = 0

    # Compute precision, recall, and F1 score for each query
    for query_id in run.keys():
        retrieved_results = run[query_id]
        relevant_results = relevance_judgements[query_id]
        relevant_and_retrieved = set(retrieved_results) & set(relevant_results)

        # Update global counts
        global_retrieved += len(retrieved_results)
        global_relevant += len(relevant_results)
        global_retrieved_and_relevant += len(relevant_and_retrieved)

        # Compute precision and recall
        precision = len(relevant_and_retrieved) / len(retrieved_results) if len(retrieved_results) > 0 else 0
        recall = len(relevant_and_retrieved) / len(relevant_results) if len(relevant_results) > 0 else 0

        # Compute F1 score if both precision and recall are non-zero
        if (precision + recall) > 0:
            f1 = 2 * (precision * recall) / (precision + recall)
            f1_values.append(f1)

        # Append precision and recall for the current query
        precision_values.append(precision)
        recall_values.append(recall)

    # Compute macro-averages
    macro_average_precision = sum(precision_values) / len(precision_values) if precision_values else 0
    macro_average_recall = sum(recall_values) / len(recall_values) if recall_values else 0
    macro_average_f1 = sum(f1_values) / len(f1_values) if f1_values else 0

    # Print macro-averaged scores
    print(f"Macro-averaged Precision: {round(macro_average_precision,3)}")
    print(f"Macro-averaged Recall: {round(macro_average_recall,3)}")
    print(f"Macro-averaged F1: {round(macro_average_f1,3)}")
    print("")

    # Compute micro-averages
    micro_average_precision = global_retrieved_and_relevant / global_retrieved if global_retrieved > 0 else 0
    micro_average_recall = global_retrieved_and_relevant / global_relevant if global_relevant > 0 else 0
    micro_average_f1 = (2 * (micro_average_precision * micro_average_recall) /
                        (micro_average_precision + micro_average_recall)) if (micro_average_precision + micro_average_recall) > 0 else 0

    # Print micro-averaged scores
    print(f"Micro-averaged Precision: {round(micro_average_precision,3)}")
    print(f"Micro-averaged Recall: {round(micro_average_recall,3)}")
    print(f"Micro-averaged F1: {round(micro_average_f1,3)}")

In [13]:
bm25_variants = ['robertson', 'bm25l', 'bm25+', 'atire', 'lucene']
stopwords_options = [True, False]
stemming_options = [True, False]

original_queries = all_queries["queries.jsonl"]

In [14]:
for variant in bm25_variants:
    for stopwords in stopwords_options:
        for stemming in stemming_options:
            stopwords_arg = "en" if stopwords else None
            stemming_arg = stemmer if stemming else None
            print()
            print(f"Variant: {variant}, Stopwords: {stopwords_arg}, Stemming: {stemming_arg}")
            corpus_tokenized = bm25s.tokenize(corpus_plaintext,stopwords=stopwords_arg,
                                              stemmer=stemming_arg,
                                              show_progress=True)
            retriever = bm25s.BM25(corpus=corpus_verbatim, method=variant, idf_method=variant)
            retriever.index(corpus_tokenized, show_progress=True)
            filename = f"{variant}-{'' if stopwords else 'NON-'}stopwords-{'' if stemming else 'NON-'}stemming.json"
            run = submit_queries_and_get_run(filename,original_queries, stemmer, retriever)
            compute_precision_recall_f1(run, relevance_judgements)




Variant: robertson, Stopwords: en, Stemming: <Stemmer.Stemmer object at 0x7d555408bb80>


Split strings:   0%|          | 0/171332 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/171332 [00:00<?, ?it/s]

DEBUG:bm25s:Building index from IDs objects


BM25S Count Tokens:   0%|          | 0/171332 [00:00<?, ?it/s]

...

Split strings:   0%|          | 0/171332 [00:00<?, ?it/s]

DEBUG:bm25s:Building index from IDs objects


BM25S Count Tokens:   0%|          | 0/171332 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/171332 [00:00<?, ?it/s]

Macro-averaged Precision: 0.341
Macro-averaged Recall: 0.029
Macro-averaged F1: 0.054

Micro-averaged Precision: 0.341
Micro-averaged Recall: 0.026
Micro-averaged F1: 0.048
